## Fundemental Hypotheses

Overall, I draw 4 hypotheses:
1) If (i) "no label" can predict sensitive features with high scores and (ii) "yes label" scores similarly to "no label", then "fairness through unawareness" will not work.

    - All we can say is that "fairness through unawareness" will not work because we can predict the sensitive features using not sensitive features.
    
2) If (i) "no label" cannot predict sensitive features well and (ii) "yes label" scores similarly to "no label", then a learning-based algorithm should already be fair regardless of "fairness through unawareness".

    - If one cannot predict the sensitive features using other features even with labels, then the labels is not correlated to the features at all. One should suspect "malicous intent" in this scenario if an algorithm has high AOD/EOD scores.
    
3) If (i) no label can predict sensitive features with high scores and (ii) "yes label" scores even higher than "no label", then "fairness through unawareness" may help to make the algorithm more fair by a little bit.

    - In this case, there are no good solutions. "Fairness through unawareness" may help a little but chances are AOD and EOD scores will still be high. 
    
4) If (i) no label cannot predict sensitive features well and (ii) "yes label" scores higher than "no label", then "fairness through unawareness" may help to make the algorithm more fair.

    - In this case, there is substantial correlation between label and gender. However, this correlation cannot be predicted or substituted using other sensitive features. Therefore, we expect "fairness through unawareness" to work well. 

In [ ]:
import os
os.chdir('../')
import sys
sys.path.append("./subjects/")

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
model = "LR"
dataset = "census"
sensitive_param = "sex"
from sklearn.covariance import EmpiricalCovariance
import sklearn.decomposition
import ast
import sys
import numpy as np
import os
# import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler

from adf_data.census import census_data
from adf_data.credit import credit_data
from adf_data.bank import bank_data
from adf_data.compas import compas_data

In [42]:
import pandas as pd
df = pd.read_csv("./Dataset" + "/" + f"{model}_{dataset}_{sensitive_param}_x100.csv")

## Without the label as a feature

Here, we are attempting to use the other features in the dataset to label the gender of a person.

In [45]:
data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas": compas_data}

X, Y, input_shape, nb_classes = data[dataset]()
Y = X[:,8]
X = np.delete(X, 8, 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [44]:
lr = LogisticRegression(max_iter=10000).fit(X_train, y_train)
lr.score(X_test,y_test)

0.709372312983663

In [21]:
dt = DecisionTreeClassifier().fit(X_train, y_train)
dt.score(X_test,y_test)

0.7995332268763051

In [36]:
sum(Y)/len(Y)

0.6692054912318418

## With the label as a feature
Here we do the same -- trying to label each person's gender. However, we add, as a feature, the original label.

In [32]:
data = {"census":census_data, "credit":credit_data, "bank":bank_data, "compas": compas_data}

X, Y, input_shape, nb_classes = data[dataset]()
Y_feature = np.argmax(Y, axis=1)
Y = X[:,8]
X = np.delete(X, 8, 1)
X = np.insert(X,0, Y_feature, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

In [34]:
lr = LogisticRegression(max_iter=10000).fit(X_train, y_train)
lr.score(X_test,y_test)

0.7112148384719322

In [35]:
dt = DecisionTreeClassifier().fit(X_train, y_train)
dt.score(X_test,y_test)

0.802972607787741

We first draw a few factual statements:

Fact 1) A learning algorithm, especially a decision tree as compared to a Logistic Regression model, can discrimate someone's gender with ~80% acuracy.

Fact 2) Adding the label does not significantly increase the model's ability to discrinate someone's gender.

From these statements, we make the following conjectures:

Conjecture 1) From Fact 1, I think it is intuitive to hypothesize that "fairness through unawareness" may not work well since an algorithm can essentially predict someone's gender anyways.

Conjecture 2) From Fact 2, I think that it is intuitive to hypothesize that the features themselves are good enough to be used to predict the gender attribute. Therefore again, "fairness through unawareness" may not work well.